In [17]:
!pip install supervision
!pip install roboflow
!pip install pybboxes

In [38]:
import json
import os
from PIL import Image
import pybboxes as pbx
import yaml
import shutil
import random
from collections import Counter

In [7]:
!mkdir -p /data/coco/

# Download the annotation ZIP and extract it, then remove the ZIP file
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d /data/coco/
!rm annotations_trainval2017.zip

# Download the train2017 ZIP, extract it, and remove the ZIP file
# !wget http://images.cocodataset.org/zips/train2017.zip
# !unzip -q train2017.zip -d /data/coco/
# !rm train2017.zip

# Download the val2017 ZIP, extract it, and remove the ZIP file
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip -d /data/coco/
!rm val2017.zip

--2024-08-14 11:44:50--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.59.33, 3.5.29.126, 3.5.27.62, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.59.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  17.7MB/s    in 46s     

2024-08-14 11:45:37 (17.0 MB/s) - ‘val2017.zip’ saved [815585330/815585330]



In [4]:
# git for cleaning coco dataset
!git clone https://github.com/immersive-limit/coco-manager.git

Cloning into 'coco-manager'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 2), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), done.
Resolving deltas: 100% (2/2), done.


In [6]:
!python /content/coco-manager/filter.py --input_json /data/coco/annotations/instances_val2017.json --output_json /data/coco/annotations/filtered_val.json --categories person dog cat

Loading json file...
Processing input json...
Filtering...
Saving new json file...
Filtered json saved.


In [20]:
with open('/data/coco/annotations/filtered_val.json', 'r') as f:
    data = json.load(f)

categories = data['categories']
category_id_to_name = {category['id']: category['name'] for category in categories}

for category_id, category_name in category_id_to_name.items():
    print(f"ID: {category_id}, Name: {category_name}")


# Print a few sample annotations
# for i, annotation in enumerate(data['annotations']):
#     print(f"Annotation {i+1}:")
#     print(json.dumps(annotation, indent=4))
#     print()
#     if i == 1:
#         break


ID: 1, Name: person
ID: 2, Name: cat
ID: 3, Name: dog


In [10]:
with open('/data/coco/annotations/instances_val2017.json', 'r') as f:
    data = json.load(f)

categories = data['categories']
category_id_to_name = {category['id']: category['name'] for category in categories}

for category_id, category_name in category_id_to_name.items():
    print(f"ID: {category_id}, Name: {category_name}")


ID: 1, Name: person
ID: 2, Name: bicycle
ID: 3, Name: car
ID: 4, Name: motorcycle
ID: 5, Name: airplane
ID: 6, Name: bus
ID: 7, Name: train
ID: 8, Name: truck
ID: 9, Name: boat
ID: 10, Name: traffic light
ID: 11, Name: fire hydrant
ID: 13, Name: stop sign
ID: 14, Name: parking meter
ID: 15, Name: bench
ID: 16, Name: bird
ID: 17, Name: cat
ID: 18, Name: dog
ID: 19, Name: horse
ID: 20, Name: sheep
ID: 21, Name: cow
ID: 22, Name: elephant
ID: 23, Name: bear
ID: 24, Name: zebra
ID: 25, Name: giraffe
ID: 27, Name: backpack
ID: 28, Name: umbrella
ID: 31, Name: handbag
ID: 32, Name: tie
ID: 33, Name: suitcase
ID: 34, Name: frisbee
ID: 35, Name: skis
ID: 36, Name: snowboard
ID: 37, Name: sports ball
ID: 38, Name: kite
ID: 39, Name: baseball bat
ID: 40, Name: baseball glove
ID: 41, Name: skateboard
ID: 42, Name: surfboard
ID: 43, Name: tennis racket
ID: 44, Name: bottle
ID: 46, Name: wine glass
ID: 47, Name: cup
ID: 48, Name: fork
ID: 49, Name: knife
ID: 50, Name: spoon
ID: 51, Name: bowl
ID: 5

In [13]:

with open('/data/coco/annotations/filtered_val.json', 'r') as f:
    filtered_data = json.load(f)

filtered_image_ids = {annotation['image_id'] for annotation in filtered_data['annotations']}

filtered_images = [image for image in filtered_data['images'] if image['id'] in filtered_image_ids]

filtered_dataset = {
    'images': filtered_images,
    'annotations': filtered_data['annotations'],
    'categories': filtered_data['categories']
}



In [49]:
def remove_unlisted_images(image_dir, filtered_dataset):
    """
    Removes all images from the specified directory that are not listed in the filtered dataset (after class filtration).

    Parameters:
    - image_dir (str): The directory containing the images to be filtered.
    - filtered_dataset (dict): Dictionary containing filtered 'images', 'annotations', and 'categories'.

    Description:
    This function scans the provided image directory and removes any image files that are not
    present in the `filtered_dataset['images']` list.
    """

    retained_images = {image_info['file_name'] for image_info in filtered_dataset['images']}

    all_images = set(os.listdir(image_dir))

    unlisted_images = all_images - retained_images

    for image_filename in unlisted_images:
        image_path = os.path.join(image_dir, image_filename)
        os.remove(image_path)

    print(f"Cleanup complete. {len(unlisted_images)} unlisted images have been removed.")



In [50]:
# Remove unlisted images
remove_unlisted_images("/data/coco/val2017", filtered_dataset)

Cleanup complete. 2055 unlisted images have been removed.


In [32]:
filtered_dataset['categories']

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'animal', 'id': 2, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 3, 'name': 'dog'}]

In [48]:
len(filtered_dataset['images'])

2945

In [16]:
from roboflow import Roboflow
rf = Roboflow(api_key="API_key")
project = rf.workspace("atathamuscoinsdataset").project("u.s.-coins-dataset-a.tatham")
dataset = project.version(5).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to U.S.-Coins-Dataset---A.Tatham-5 in yolov5pytorch:: 100%|██████████| 3306/3306 [00:01<00:00, 1666.18it/s]


In [31]:
def coco_to_yolo(coco_annotation_path, images_dir, output_dir, yaml_output_path=None):
    """
    Converts COCO format annotations to YOLO format and generates a YAML file describing the dataset.

    Parameters:
    - coco_annotation_path (str): Path to the COCO annotation JSON file.
    - images_dir (str): Directory containing the images referenced in the COCO annotations.
    - output_dir (str): Directory where YOLO format annotation files will be saved.
    - yaml_output_path (str, optional): Path to save the generated YAML file. If None, it will save in the `output_dir`.

    Description:
    This function reads annotations from a COCO format JSON file and converts them to the YOLO format.
    It also generates a `data.yaml` file that describes the dataset.
    In YOLO format, each image has a corresponding .txt file containing the bounding box annotations.
    Each line in the .txt file follows the format:

    class_id center_x center_y width height

    Where:
    - `class_id` is the ID of the object class (starting from 0).
    - `center_x`, `center_y` are the normalized coordinates of the bounding box center.
    - `width`, `height` are the normalized dimensions of the bounding box.

    The coordinates and dimensions are normalized with respect to the image width and height.

    The function processes each annotation in the COCO dataset, converts the bounding box to YOLO format
    using the `pybboxes` library, and saves the results in a .txt file named after the corresponding image.

    """

    class_names = [category['name'] for category in coco_data['categories']]

    with open(coco_annotation_path, 'r') as f:
        coco_data = json.load(f)

    os.makedirs(output_dir, exist_ok=True)

    category_id_to_yolo_id = {category['id']: idx for idx, category in enumerate(coco_data['categories'])}

    # Process each annotation
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        category_id = annotation['category_id']
        bbox = annotation['bbox']

        # Get image file name and size
        image_info = next(image for image in coco_data['images'] if image['id'] == image_id)
        image_filename = image_info['file_name']
        image_path = os.path.join(images_dir, image_filename)

        if not os.path.exists(image_path):
            print(f"Warning: Label file not found for image: {image_path}")
            continue

        with Image.open(image_path) as img:
            image_size = img.size  # (width, height)

        yolo_bbox = pbx.convert_bbox(bbox, from_type="coco", to_type="yolo", image_size=image_size)

        yolo_class_id = category_id_to_yolo_id[category_id]

        yolo_annotation_line = f"{yolo_class_id} {' '.join(map(str, yolo_bbox))}\n"

        yolo_annotation_file = os.path.join(output_dir, f"{os.path.splitext(image_filename)[0]}.txt")

        with open(yolo_annotation_file, 'a') as yolo_file:
            yolo_file.write(yolo_annotation_line)

    yaml_content = {
        'nc': len(class_names),                     # Number of classes
        'names': class_names                        # List of class names
    }

    # Determine where to save the YAML file
    if yaml_output_path is None:
        yaml_output_path = os.path.join(output_dir, 'data.yaml')

    # Save the YAML file
    with open(yaml_output_path, 'w') as yaml_file:
        yaml.dump(yaml_content, yaml_file)

    print(f"YOLO annotations and data.yaml have been saved to {output_dir}")

{1: 0, 2: 1, 3: 2}


KeyboardInterrupt: 

In [ ]:
# clean folders
!rm -r /data/coco/annotations/yolo_format/

In [ ]:
coco_to_yolo(
    coco_annotation_path='/data/coco/annotations/filtered_val.json',
    images_dir='/data/coco/val2017',
    output_dir='/data/coco/annotations/yolo_format/'
)

In [36]:
def create_dataset(filtered_dataset, dataset_params, input_params, split_ratios=(0.7, 0.2, 0.1), move_files=False):
    """
    Create a dataset structure for training, validation, and test sets with images and YOLO labels.

    Parameters:
    - filtered_dataset (dict): Dictionary containing filtered 'images', 'annotations', and 'categories'.
    - dataset_params (dict): Dictionary defining the dataset structure and paths.
    - input_params (dict): Dictionary defining the input structure with paths to images and labels.
    - split_ratios (tuple): Ratios for splitting the dataset into (train, valid, test).
    - move_files (bool): Whether to move files instead of copying.

    Description:
    This function divides images into train, validation, and test sets based on the provided split ratios.
    Labels has to be first processed by coco_to_yolo function. It then copies or moves the images
    and corresponding YOLOv5 format label files from the input directories to the designated
    output folders as per the structure defined in `dataset_params`.
    """

    for dir_path in [dataset_params['train_images_dir'], dataset_params['train_labels_dir'],
                     dataset_params['val_images_dir'], dataset_params['val_labels_dir'],
                     dataset_params['test_images_dir'], dataset_params['test_labels_dir']]:
        full_path = os.path.join(dataset_params['data_dir'], dir_path)
        os.makedirs(full_path, exist_ok=True)

    images = filtered_dataset['images']
    random.shuffle(images)

    train_idx = int(len(images) * split_ratios[0])
    val_idx = train_idx + int(len(images) * split_ratios[1])

    train_images = images[:train_idx]
    val_images = images[train_idx:val_idx]
    test_images = images[val_idx:]

    def process_images(image_set, image_dir, label_dir):
        for image_info in image_set:
            image_filename = image_info['file_name']
            yolo_label_filename = os.path.splitext(image_filename)[0] + '.txt'

            image_src_path = os.path.join(input_params['images_dir'], image_filename)
            label_src_path = os.path.join(input_params['labels_dir'], yolo_label_filename)

            image_dest_path = os.path.join(dataset_params['data_dir'], image_dir, image_filename)
            label_dest_path = os.path.join(dataset_params['data_dir'], label_dir, yolo_label_filename)

            # Copy or move files
            if move_files:
                shutil.move(image_src_path, image_dest_path)
                shutil.move(label_src_path, label_dest_path)
            else:
                shutil.copy(image_src_path, image_dest_path)
                shutil.copy(label_src_path, label_dest_path)

    # Process each set
    process_images(train_images, dataset_params['train_images_dir'], dataset_params['train_labels_dir'])
    process_images(val_images, dataset_params['val_images_dir'], dataset_params['val_labels_dir'])
    process_images(test_images, dataset_params['test_images_dir'], dataset_params['test_labels_dir'])

    print(f"Dataset created in {dataset_params['data_dir']} with train/val/test splits.")

In [39]:
dataset_params = {
    'data_dir': '/content/dataset_train',
    'train_images_dir': 'train/images',
    'train_labels_dir': 'train/labels',
    'val_images_dir': 'valid/images',
    'val_labels_dir': 'valid/labels',
    'test_images_dir': 'test/images',
    'test_labels_dir': 'test/labels',
    'classes': ['person', 'cat', 'dog']
}

input_params = {
    'images_dir': '/data/coco/val2017',
    'labels_dir': '/data/coco/annotations/yolo_format'
}

split_ratios = (0.7, 0.2, 0.1)

create_dataset(filtered_dataset, dataset_params, input_params, split_ratios=split_ratios, move_files=False)

## Test

In [46]:
def test_dataset_creation(input_params, dataset_params, split_ratios):

    # List all files in the input directories
    input_images = set(os.listdir(input_params['images_dir']))
    input_labels = set(os.listdir(input_params['labels_dir']))

    # List all files in the output directories
    train_images = set(os.listdir(os.path.join(dataset_params['data_dir'], dataset_params['train_images_dir'])))
    train_labels = set(os.listdir(os.path.join(dataset_params['data_dir'], dataset_params['train_labels_dir'])))

    val_images = set(os.listdir(os.path.join(dataset_params['data_dir'], dataset_params['val_images_dir'])))
    val_labels = set(os.listdir(os.path.join(dataset_params['data_dir'], dataset_params['val_labels_dir'])))

    test_images = set(os.listdir(os.path.join(dataset_params['data_dir'], dataset_params['test_images_dir'])))
    test_labels = set(os.listdir(os.path.join(dataset_params['data_dir'], dataset_params['test_labels_dir'])))

    # Check if all images and labels were moved or copied
    output_images = train_images | val_images | test_images
    output_labels = train_labels | val_labels | test_labels

    assert input_images == output_images, f"Mismatch in images. Missing images: {input_images - output_images}"
    assert input_labels == output_labels, f"Mismatch in labels. Missing labels: {input_labels - output_labels}"

    # Check proportions
    total_images = len(input_images)
    total_labels = len(input_labels)

    assert len(train_images) == len(train_labels),
    assert len(val_images) == len(val_labels),
    assert len(test_images) == len(test_labels),

    expected_train_size = int(total_images * split_ratios[0])
    expected_val_size = int(total_images * split_ratios[1])
    expected_test_size = total_images - expected_train_size - expected_val_size

    assert len(train_images) == expected_train_size, f"Training set size is incorrect. Expected {expected_train_size}, got {len(train_images)}"
    assert len(val_images) == expected_val_size, f"Validation set size is incorrect. Expected {expected_val_size}, got {len(val_images)}"
    assert len(test_images) == expected_test_size, f"Test set size is incorrect. Expected {expected_test_size}, got {len(test_images)}"

    print("All tests passed successfully.")


In [51]:
test_dataset_creation(input_params, dataset_params, split_ratios)

2945 2061
All tests passed successfully.
